In [1]:
# Basic packages always been used
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Data scaling
from sklearn.preprocessing import MinMaxScaler

# Function for spilting training & testing data set
from sklearn.model_selection import train_test_split

# Tensorflow sequential models
from tensorflow import keras
from keras import backend as clear
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.optimizers import SGD
from keras.optimizers import Adam

# Functions for evaluation
from sklearn.metrics import mean_absolute_error #MAE
from sklearn.metrics import mean_squared_error #MSE,RMSE
from sklearn.metrics import mean_absolute_percentage_error #MAPE

# Note: Some metrics need to be defined manually

In [2]:
# Symmetric Mean Absolute Percentage Error (SMAPE)
def SMAPE_calulate(y_true, y_pred):
    n = len(y_true)
    SMAPE=(100 / n) * np.sum(2 * np.abs(y_pred - y_true) / (np.abs(y_pred) + np.abs(y_true)))
    return SMAPE

# Relative Absolute Error (RAE)
def RAE_calculate(y_true, y_pred):
    abs_errors = np.abs(y_true - y_pred)
    denominator = np.sum(np.abs(y_true - np.mean(y_true)))
    RAE=np.sum(abs_errors/denominator)
    return RAE

# Mean Relative Absolute Error (MRAE)
def MRAE_calculate(y_true, y_pred):
    n=len(y_true)
    abs_errors = np.abs(y_true - y_pred)
    denominator = np.sum(np.abs(y_true - np.mean(y_true)))
    RAE=np.sum(abs_errors/denominator)
    MRAE=RAE/n
    return MRAE

# Median Relative Absolute Error (MdRAE) comparing with benchmark
# Note: By default, the bench value is the mean of actual value
def MdRAE_calculate(y_true, y_pred, bench=None):
    if bench==None:
        bench=np.mean(y_true)
        MdRAE=np.median(np.abs(y_true - y_pred)/np.abs(y_true - bench))
        return MdRAE,bench
    else:
        MdRAE=np.median(np.abs(y_true - y_pred)/np.abs(y_true - bench))
        return MdRAE
    
# Relative Squared Error (RSE) & Root Relative Squared Error (RRSE)
def RSE_calculate(y_true, y_pred, Root=False):
    mse = np.mean((y_true - y_pred)**2)
    denominator = np.var(y_true)
    RSE=mse/denominator
    
    if Root==True:
        return np.sqrt(RSE)
    else:
        return RSE

In [3]:
#load data(sometimes need to use "/")
df = pd.read_csv('/content/House_Rent_Dataset.csv')
# Colab
#df = pd.read_csv('')
df

,Rent,Size,BHK,Bathroom,Furnishing Status,City
0,10000,1100,2,2,Unfurnished,Kolkata
1,20000,800,2,1,Semi-Furnished,Kolkata
2,17000,1000,2,1,Semi-Furnished,Kolkata
3,10000,800,2,1,Unfurnished,Kolkata
4,7500,850,2,1,Unfurnished,Kolkata
...,...,...,...,...,...,...
4741,15000,1000,2,2,Semi-Furnished,Hyderabad
4742,29000,2000,3,3,Semi-Furnished,Hyderabad
4743,35000,1750,3,3,Semi-Furnished,Hyderabad
4744,45000,1500,3,2,Semi-Furnished,Hyderabad


# 1. Quick overview to get a grasp of the data set

key function: pd.info(); pd.astype(); describe(); pd.value_counts()

In [4]:
# 1.1 Easiest way to check data type and if there are any missing value
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4746 entries, 0 to 4745
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Rent               4746 non-null   int64 
 1   Size               4746 non-null   int64 
 2   BHK                4746 non-null   int64 
 3   Bathroom           4746 non-null   int64 
 4   Furnishing Status  4746 non-null   object
 5   City               4746 non-null   object
dtypes: int64(4), object(2)
memory usage: 222.6+ KB


In [5]:
# 1.2 When using the describe function in default, pandas automatically selects the numerical columns
df.describe()

,Rent,Size,BHK,Bathroom
count,4.746000e+03,4746.000000,4746.000000,4746.000000
mean,3.499345e+04,967.490729,2.083860,1.965866
std,7.810641e+04,634.202328,0.832256,0.884532
min,1.200000e+03,10.000000,1.000000,1.000000
25%,1.000000e+04,550.000000,2.000000,1.000000
50%,1.600000e+04,850.000000,2.000000,2.000000
75%,3.300000e+04,1200.000000,3.000000,2.000000
max,3.500000e+06,8000.000000,6.000000,10.000000


Note: Pclass is categorical data with dummies, so converting to correct data type is required

In [6]:
# 1.3 Using value_counts function to count frequency in categorical column
print(df[['Furnishing Status']].value_counts(sort=True))
print('+-----------------------+')
print(df[['City']].value_counts(sort=True))
print('+-----------------------+')

Furnishing Status
Semi-Furnished       2251
Unfurnished          1815
Furnished             680
dtype: int64
+-----------------------+
City     
Mumbai       972
Chennai      891
Bangalore    886
Hyderabad    868
Delhi        605
Kolkata      524
dtype: int64
+-----------------------+


# 2. Data preprocessing

key function: pd.dropma() unique(); LabelEncoder(); pd.get_dummies

In [7]:
#2.1 Using pd.get_dummies function to generate dummies
dummied_df=pd.get_dummies(df,columns=['Furnishing Status','City'])
dummied_df

,Rent,Size,BHK,Bathroom,Furnishing Status_Furnished,Furnishing Status_Semi-Furnished,Furnishing Status_Unfurnished,City_Bangalore,City_Chennai,City_Delhi,City_Hyderabad,City_Kolkata,City_Mumbai
0,10000,1100,2,2,0,0,1,0,0,0,0,1,0
1,20000,800,2,1,0,1,0,0,0,0,0,1,0
2,17000,1000,2,1,0,1,0,0,0,0,0,1,0
3,10000,800,2,1,0,0,1,0,0,0,0,1,0
4,7500,850,2,1,0,0,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4741,15000,1000,2,2,0,1,0,0,0,0,1,0,0
4742,29000,2000,3,3,0,1,0,0,0,0,1,0,0
4743,35000,1750,3,3,0,1,0,0,0,0,1,0,0
4744,45000,1500,3,2,0,1,0,0,0,0,1,0,0


Note: get_dummies function generates the onehotencode style dummies

In [8]:
# Export to CSV file Note: preset folder path is required
#dummied_df.to_csv('/content/House_Rent_Dataset.csv',index=False, header=True)
# Colab
#dummied_new_df.to_csv('House_Rent_dummied.csv',index=False, header=True)
print('Export complete...')

Export complete...


# 3. Set dependent variable(Y or target) & independent variable(X) 

In [9]:
#set x (make prediction) with minimax
x=dummied_df.drop(['Rent'],axis=1).copy()

#set y (want to predict)
y=dummied_df['Rent'].copy()

Note: Target variable Y does not need to be scaled, so we  need to separate dependent variabe and independent variables first

# 4. Data scaling with minimax technique

Key function: MinMaxScaler()

In [10]:
#minimax scaling
MMscaler=MinMaxScaler(feature_range=(0, 1))
scaling=MMscaler.fit_transform(x)
scaled_x=pd.DataFrame(data=scaling)
scaled_x.columns=x.columns
scaled_x.head()

,Size,BHK,Bathroom,Furnishing Status_Furnished,Furnishing Status_Semi-Furnished,Furnishing Status_Unfurnished,City_Bangalore,City_Chennai,City_Delhi,City_Hyderabad,City_Kolkata,City_Mumbai
0,0.136421,0.2,0.111111,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.098874,0.2,0.000000,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.123905,0.2,0.000000,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.098874,0.2,0.000000,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.105131,0.2,0.000000,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


# 5. Spilt the data into train & test set

key function: train_test_split()

key parameters for train_test_spilt(): test_size; random_state 

In [11]:
#prepare dataset with scaling
#Split the data (split into 80% training data & 20% testing data) (lock seed)
x_train,x_test,y_train,y_test=train_test_split(scaled_x,y,test_size=0.2,random_state=444)

In [12]:
# 
print('Amount of training samples:',len(y_train))
print('+----------------------------------------+')
print('Amount of testing samples:',len(y_test))

Amount of training samples: 3796
+----------------------------------------+
Amount of testing samples: 950


# 6. Creating Net 

In [13]:
# Quick recap: Total of 12 features in the data set
scaled_x.head()

,Size,BHK,Bathroom,Furnishing Status_Furnished,Furnishing Status_Semi-Furnished,Furnishing Status_Unfurnished,City_Bangalore,City_Chennai,City_Delhi,City_Hyderabad,City_Kolkata,City_Mumbai
0,0.136421,0.2,0.111111,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.098874,0.2,0.000000,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.123905,0.2,0.000000,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.098874,0.2,0.000000,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.105131,0.2,0.000000,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


In [14]:
# Tips: Using shape to call the amount of features in the data set 
input_features=scaled_x.shape[1]
print('Amount of features:',input_features)

Amount of features: 12


# Important: Define customized loss function (RMSE)

In [15]:
# Some self define functions for building deep neural networks
def dense_generator(model_name,layers,last_layer_neurons,activation_fun='relu'):
    n=last_layer_neurons
    neurons=n
    for i in range(0,layers):
        neurons=neurons*2
        model_name.add(Dense(units=neurons, activation=activation_fun))

Note: As far as I know when I'm preparing this in-class material, keras does not have a built-in RMSE function. Rumor has it that a RMSE function is hidden in the latest version, however it needs to be confirmed. For instance, defining RMSE ourself is not difficult, just don't forget to register our custom functions.   

# Sequential style coding for creating net

In [16]:
# with tf.device(device_name):
# 6.1 Create the model
clear.clear_session()
model=Sequential()

# 6.2 Add input layer & first hidden layer
neurons=24
for i in range(0,10):
    neurons=neurons*2
    model.add(Dense(units=neurons, input_dim=12, activation='relu'))
    if (i%3==0):
        model.add(Dropout(rate=0.2))


# 6.4 Add dropout layer




# 6.6 Add regularization layer optional
#model.add(BatchNormalization())

# 6.7 Add output softmax layer
model.add(Dense(units=1, activation='linear'))

# 6.8 Compile the defined Net
#opt = SGD(learning_rate=0.0001, momentum=0.9)
opt=Adam(learning_rate=0.00005,beta_1=0.9)
model.compile(loss='mean_squared_error', optimizer=opt, metrics=['mean_absolute_percentage_error'])

# Finally check the model 
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 48)                624       
                                                                 
 dropout (Dropout)           (None, 48)                0         
                                                                 
 dense_1 (Dense)             (None, 96)                4704      
                                                                 
 dense_2 (Dense)             (None, 192)               18624     
                                                                 
 dense_3 (Dense)             (None, 384)               74112     
                                                                 
 dropout_1 (Dropout)         (None, 384)               0         
                                                                 
 dense_4 (Dense)             (None, 768)               2

Note: Remember to clear the session or else some of the left neurans may cause trouble while fitting

# 7. Fit the model

# Note: The validation set should be split manually

In [ ]:
# Important
x_training,x_validation,y_training,y_validation=train_test_split(x_train,y_train,test_size=0.2,random_state=444)

# 7.1 Store in the history for more useful information
history = model.fit(x_training, y_training, epochs=100, batch_size=128, verbose=1, validation_data=(x_validation, y_validation))

training_mape = history.history['mean_absolute_percentage_error']
val_mape = history.history['val_mean_absolute_percentage_error']


Epoch 1/100
20/24 [========================>.....] - ETA: 1:18 - loss: 8948192256.0000 - mean_absolute_percentage_error: 99.6724

In [ ]:
# Check the dictionary keys
modeling_result=history.history
modeling_result.keys()

In [ ]:
# 7.2 Plot the history of training and validation
training_loss_values = history.history['val_loss']
val_loss_values = history.history['loss']
epochs = range(1, len(training_loss_values) + 1)

plt.figure(figsize=(12, 10), dpi=300)
plt.xlabel('Epochs', fontsize=20)
plt.ylabel('Loss (Root Mean Squared Error, RMSE)', fontsize=20)
plt.title('House Rent DNN training & validation of Loss result', fontsize=20)

plt.plot(epochs, training_loss_values, marker='o', label='training Loss')
plt.plot(epochs, val_loss_values, marker='o', label='validation Loss')
plt.legend(loc=1, fontsize=24)
plt.show()


In [ ]:
# 7.2 Plot the history of training and validation
training_acc_values=modeling_result['val_mean_absolute_percentage_error']
val_acc_values=modeling_result['mean_absolute_percentage_error']
epochs=range(1,len(training_acc_values)+1)

plt.figure(figsize=(12,10),dpi=300)
plt.xlabel('Epochs',fontsize=20)
plt.ylabel('Accuracy',fontsize=20)
plt.title('House Rent DNN training & validation of MAPE result ',fontsize=20)

plt.plot(epochs, training_acc_values,marker='o',label='training MAPE')
plt.plot(epochs, val_acc_values,marker='o',label='validation MAPE')
plt.legend(loc=1,fontsize=24)
plt.show()

In [ ]:
# 7.3 Save the trained model
model.save('D:/data_analysis/PyExport/House_Rent_ANN_trained_model.h5')
# Colab
#model.save('House_Rent_ANN_trained_model.h5')
print('Model has been saved...')

In [ ]:
# 7.4 Restore the saved model for testing
ANN_model=keras.models.load_model('D:/data_analysis/PyExport/House_Rent_ANN_trained_model.h5')
# Colab
#ANN_model=keras.models.load_model('')
print('Model successfully loaded...')

# 8. Testing

In [ ]:
# 8.1 Make prediction 
prediction=np.round(ANN_model.predict_on_batch(x_test),0)
prediction=prediction.astype('int')
pred_values=[]
for i in range(0,len(prediction)):
    value=prediction[i][0]
    pred_values.append(value)

Note: Since Rent value is integer, we need to transform the outputs and convert the data type.

In [ ]:
Results=pd.DataFrame({'Num':range(1,len(prediction)+1),'Y_true':y_test,'Predict':pred_values})
Results['abs Error']=np.abs(Results['Y_true']-Results['Predict'])
Results

In [ ]:
# 8.2 Calculating the mertics
MAE=mean_absolute_error(y_test,pred_values)
MSE=mean_squared_error(y_test,pred_values,squared=True)
RMSE=mean_squared_error(y_test,pred_values,squared=False)
MAPE=mean_absolute_percentage_error(y_test,pred_values)
SMAPE=SMAPE_calulate(y_test,pred_values)
RAE=RAE_calculate(y_test,pred_values)
MRAE=MRAE_calculate(y_test,pred_values)
MdRAE=MdRAE_calculate(y_test,pred_values)
RSE=RSE_calculate(y_test,pred_values,Root=False)
RRSE=RSE_calculate(y_test,pred_values,Root=True)

print('MAE:',round(MAE,3))
print('MSE:',round(MSE,3))
print('RMSE:',round(RMSE,3))
print('MAPE:',round(MAPE*100,3),'%')
print('SMAPE:',round(SMAPE,3),'%')
print('RAE:',round(RAE,3))
print('MRAE:',MRAE)
print('MdRAE:',round(MdRAE[0],3),'bench:',round(MdRAE[1],3))
print('RSE:',round(RSE,3))
print('RRSE:',round(RRSE,3))

# Scatter plot for comparing true value & predicted value 

In [ ]:
# Figure setting
plt.figure(figsize=(30,8),dpi=600)
plt.title('True value vs Predict value',weight='bold',fontsize=36)
plt.xlabel('Observations',fontsize=24,weight='bold')
plt.ylabel('Value',fontsize=24,weight='bold')

# Scatter plot
plt.scatter(Results.iloc[:,0],Results.iloc[:,1],color='blue',label='Y_true')
plt.scatter(Results.iloc[:,0],Results.iloc[:,2],color='red',label='Y_pred')

# Figure setting
plt.legend(loc=1,fontsize=24)
# Set x-axis to let the plot looks nice
plt.xlim(-50,1000)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.show()

# Actual by predicted plot

In [ ]:
# Lock the scale of axis by Y_true
max_value=max(y_test)+100000
min_value=min(y_test)-100000

# Figure setting
plt.figure(figsize=(10,10),dpi=300)
plt.title('Actual by predicted plot',weight='bold',fontsize=24)
plt.xlabel('Y_true',fontsize=18,weight='bold')
plt.ylabel('Prediction',fontsize=18,weight='bold')

# Perfect model reference line
plt.plot([min_value,max_value], [min_value,max_value], linestyle='--',color='red')

# Actual by predicted plot
plt.scatter(Results.iloc[:,1],Results.iloc[:,2],marker='.',color='blue')

# Set the axis
plt.ylim(0,max_value)
plt.xlim(0,max_value)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.show()

In [ ]:
max_value=max(y_test)+100000
min_value=min(y_test)-100000

plt.figure(figsize=(10,10),dpi=300)
plt.title('Actual by predicted plot',weight='bold',fontsize=24)
plt.xlabel('Y_true',fontsize=18,weight='bold')
plt.ylabel('Prediction',fontsize=18,weight='bold')
plt.plot([min_value,max_value], [min_value,max_value], linestyle='--',color='red')
plt.scatter(Results.iloc[:,1],Results.iloc[:,2],marker='.',color='blue')

#plt.legend(loc=1,fontsize=18)
plt.ylim(0,100000)
plt.xlim(0,100000)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.show()